# 用例ベース対話システム（非タスク対話）
ここでは非タスク対話を対象として、用例ベースの雑談対話システムを作成します。

## 事前の設定
- 特になし

In [1]:
# 必要なラブラリを読み込む

import numpy as np
import MeCab
from gensim.models import KeyedVectors

c:\Users\inoue\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\inoue\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


ここでは、下記のような用例ベース対話システムを実装します。フレームを実装します。
あらかじめ想定されるユーザ発話とそれに対応するシステム応答のペアを複数用意しておき、ユーザ発話が入力されたら最も近い想定ユーザ発話を検索します。
そして、それに対応するシステム応答を出力します。
類似度の計算には、コサイン距離を用います。
ユーザ発話の表現には、言語理解のようにBag-of-Wordsと、学習済みWord2vecによる文ベクトルを用います。

<img src="./img/example.png" style="width: 600px;"/>

## 用例データの用意

まずは、想定されるユーザ発話とそれに対応するシステム応答のペアデータ（用例データ）を読み込みます。
データは data/example-base-data.csvに格納されており、各行が１つのペアデータ、１列目が想定ユーザ発話、２列目がシステム応答です。
ついでにMeCabによる単語分割も済ませておきます。

In [2]:
# 用例データを読み込む
pair_data = []
filename = './data/example-base-data.csv'
print('Load from %s' % filename)
with open(filename, 'r', encoding='utf8') as f:
    lines = f.readlines()
    for line in lines:
        u1 = line.split(',')[0].strip()
        u2 = line.split(',')[1].strip()
        pair_data.append([u1, u2])
        
        print('%s -> %s' % (u1, u2))

Load from ./data/example-base-data.csv
こんにちは -> こんにちは
趣味は何ですか -> 趣味はスポーツ観戦です
好きな食べ物は何ですか -> りんごです
一番安い商品は何ですか -> 一番安いのはもやしです
最近食べた料理は何ですか -> 最近食べたのはラーメンです
出身はどこですか -> 出身は京都です
印象に残っている旅行はなんですか -> 印象に残っているのはヨーロッパ旅行です
おはよう -> おはようございます
こんばんは -> こんばんは
さようなら -> さようなら
好きな芸能人は誰ですか -> 好きな芸能人はタモリです
好きなスポーツは何ですか -> 好きなスポーツはサッカーです
好きな動物は何ですか -> 好きな動物は犬です
休日は何をされていますか -> 休日は主に散歩しています
仕事は何をしていますか -> 仕事は受付係をしています
最近はまっていることは何ですか -> 最近は映画鑑賞にはまっています
好きな映画は何ですか -> 好きな映画はスターウォーズです
好きなゲームは何ですか -> 好きなゲームはポケモンです
好きなポケモンは何ですか -> 好きなポケモンはピカチュウです
好きな本は何ですか -> 好きな本は純粋理性批判です
好きな小説は何ですか -> 好きな小説は指輪物語です
好きな漫画は何ですか -> 好きな漫画はドラえもんです
好きなアニメは何ですか -> 好きなアニメはドラゴンボールです
おすすめのお店はどこですか -> おすすめのお店はサイゼリアです
得意な料理は何ですか -> チャーハンが得意料理です
好きな教科は何ですか -> 好きな教科は数学です
おすすめの観光地はどこですか -> おすすめは清水寺です
おすすめのお土産は何ですか -> おすすめは八つ橋です
好きなボードゲームは何ですか -> カタンです
好きな数字は何ですか -> 好きな数字は1です
印象に残っている映画は何ですか -> 印象に残っているのはシャイニングです
好きな季節は何ですか -> 好きな季節は夏です
嫌いな食べ物は何ですか -> 嫌いな食べ物はキウイです
嫌いな動物は何ですか -> 嫌いな動物はカラスです
何歳ですか -> 20歳です
健康にはどのように気を付けていますか -> 毎日運動するようにしています

In [3]:
# 各発話をMeCabで分割しておき、名詞・形容詞・動詞・感動詞のみを扱う
def parse_mecab(sentence):
    
    m = MeCab.Tagger ("")
    d_list = m.parse(sentence).strip().split('\n')
    
    u = []
    for d in d_list:
        
        if d.strip() == 'EOS':
            break
        
        if len(d.split('\t')) == 2:
            word = d.split('\t')[0]
            pos = d.split('\t')[1].split(',')[0]
        else:
            word = d.split('\t')[0]
            pos = d.split('\t')[4].split('-')[0]
        
        if pos in ['名詞', '形容詞', '動詞', '感動詞']:
            u.append(word)
    
    return u
    
pair_data_mecab = []
m = MeCab.Tagger ("")
for d in pair_data:
    u1 = parse_mecab(d[0])
    u2 = d[1]
    pair_data_mecab.append([u1, u2])
    
    print(u1)
    print(u2)

['こんにちは']
こんにちは
['趣味', '何']
趣味はスポーツ観戦です
['好き', '食べ物', '何']
りんごです
['一番', '安い', '商品', '何']
一番安いのはもやしです
['最近', '食べ', '料理', '何']
最近食べたのはラーメンです
['出身', 'どこ']
出身は京都です
['印象', '残っ', 'いる', '旅行', 'なん']
印象に残っているのはヨーロッパ旅行です
['おはよう']
おはようございます
['こんばんは']
こんばんは
['さようなら']
さようなら
['好き', '芸能人', '誰']
好きな芸能人はタモリです
['好き', 'スポーツ', '何']
好きなスポーツはサッカーです
['好き', '動物', '何']
好きな動物は犬です
['休日', '何', 'さ', 'れ', 'い']
休日は主に散歩しています
['仕事', '何', 'し', 'い']
仕事は受付係をしています
['最近', 'はまっ', 'いる', 'こと', '何']
最近は映画鑑賞にはまっています
['好き', '映画', '何']
好きな映画はスターウォーズです
['好き', 'ゲーム', '何']
好きなゲームはポケモンです
['好き', 'ポケモン', '何']
好きなポケモンはピカチュウです
['好き', '本', '何']
好きな本は純粋理性批判です
['好き', '小説', '何']
好きな小説は指輪物語です
['好き', '漫画', '何']
好きな漫画はドラえもんです
['好き', 'アニメ', '何']
好きなアニメはドラゴンボールです
['おすすめ', '店', 'どこ']
おすすめのお店はサイゼリアです
['得意', '料理', '何']
チャーハンが得意料理です
['好き', '教科', '何']
好きな教科は数学です
['おすすめ', '観光', '地', 'どこ']
おすすめは清水寺です
['おすすめ', '土産', '何']
おすすめは八つ橋です
['好き', 'ボード', 'ゲーム', '何']
カタンです
['好き', '数字', '何']
好きな数字は1です
['印象', '残っ', 'いる', '映画', '何']
印象に残っているのはシャイニングです
['好き', '季節', '何

次に、想定ユーザ発話を用いてBag-of-Words表現を作成します。

In [4]:
# 想定ユーザ発話を用いてBag-of-Words表現を作成する

# 学習データの想定ユーザ発話の単語を語彙（カバーする単語）とする
word_list = {}

for each_pair in pair_data_mecab:
    for word in each_pair[0]:
        word_list[word] = 1

print(word_list.keys())

# 単語とそのインデクスを作成する
word_index = {}
for idx, word in enumerate(word_list.keys()):
    word_index[word] = idx

print(word_index)

# ベクトルの次元数（未知語を扱うためにプラス１）
vec_len = len(word_list.keys()) + 1
print(vec_len)

dict_keys(['こんにちは', '趣味', '何', '好き', '食べ物', '一番', '安い', '商品', '最近', '食べ', '料理', '出身', 'どこ', '印象', '残っ', 'いる', '旅行', 'なん', 'おはよう', 'こんばんは', 'さようなら', '芸能人', '誰', 'スポーツ', '動物', '休日', 'さ', 'れ', 'い', '仕事', 'し', 'はまっ', 'こと', '映画', 'ゲーム', 'ポケモン', '本', '小説', '漫画', 'アニメ', 'おすすめ', '店', '得意', '教科', '観光', '地', '土産', 'ボード', '数字', '季節', '嫌い', '歳', '健康', 'よう', '気', '付け', 'テレビ', '番組', '絵画', '芸術', '家', '美術館', '博物館', '魚', '購入', '喫茶店', '祭り', '天気', '元素', '数式', 'プログラミング', '言語', '明日', '予定', '教え', 'ください', 'マイ', 'ブーム', 'トレンド', '絵', '上達', '方法', '対話', '仕方', '研究', '内容', '尊敬', '人', '近く', '美味しい', '動画', '配信', '見', '暑い', '実家', '普段', '音楽', '聴い', '面白い', '部', '活動', '陸上', '将来', '夢', 'いつ', 'の', '専攻', '使っ', 'パソコン', '諺', '読ん', '今度', 'あり', '来週', '忙しい', '友達', '夏休み', '過ごし', '日本', '考え', '楽しい', '充実', '応援', 'チーム', '兄弟', '欠かさ', 'ドラマ', '服', '買い', '住ん', '職場', '通勤', 'お昼', 'みんな', '呼ば', 'なる', 'ニュース', 'コロナ', '大変', '外出', 'でき', '辛い', '海外', '行っ', 'お菓子', '一人暮らし', '食事', 'ラーメン', '屋', '学生', '時代', '思い出', 'お願い'])
{'こんにちは': 0, '趣味': 1, '何': 2, '

In [5]:
# 単語の系列とBag-of-Words表現を作成するための情報を受け取りベクトルを返す関数を定義
# ※言語理解のときと同じ関数
def make_bag_of_words(words, vocab, dim, pos_unk):

    vec = [0] * dim
    for w in words:

        # 未知語
        if w not in vocab:
            vec[pos_unk] = 1
        
        # 学習データに含まれる単語
        else:
            vec[vocab[w]] = 1
    
    return vec

# 試しに変換してみる
feature_vec = make_bag_of_words(pair_data_mecab[0][0], word_index, vec_len, vec_len-1)
print(feature_vec)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 類似度計算

次に、類似度を計算する関数を用意します。
ここでは、入力ユーザ発話の単語の系列と、用例データを受け取り、入力ユーザ発話に最も類似するシステム応答を返します。

In [6]:
# 類似度計算
# 入力：ユーザ発話の単語の系列と用例データ
# 出力：入力ユーザ発話に最も類似するシステム応答
def matching_bagofwords(input_data_mecab, pair_data_mecab):
    
    # コサイン類似度が最も高いものを採用
    cos_dist_max = 0.
    response = None
    
    # 用例毎に処理
    for pair_each in pair_data_mecab:
        
        # Bag-of-Words表現に変換
        v1 = np.array(make_bag_of_words(input_data_mecab, word_index, vec_len, vec_len-1))
        v2 = np.array(make_bag_of_words(pair_each[0], word_index, vec_len, vec_len-1))
        
        # コサイン類似度を計算
        cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        if cos_dist_max < cos_sim:
            cos_dist_max = cos_sim
            response = pair_each[1]
    
    return response, cos_dist_max
        

## テスト（Bag-of-Words）
では、用例ベース対話システムをテストしてみましょう。

### Word2vecの利用
次に，特徴量としてBag-of-wordsではなくWord2vecを用いてみます。日本語でも様々な学習済みモデルがありますが、ここでは下記のものを用います。Word2vecは単語間の距離（類似性）を考慮することができるため、Bag-of-wordsよりもより頑健になることが期待されます。

- 日本語 Wikipedia エンティティベクトル
    - http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/
    - 最新のモデルをダウンロードして解凍してください

In [17]:
# テスト

# 入力発話　その１
input_data = '趣味は何ですか'
input_data_mecab = parse_mecab(input_data)

response, cos_dist_max = matching_bagofwords(input_data_mecab, pair_data_mecab)

print('入力：%s' % input_data)
print('応答：%s' % response)
print('類似度：%.3f' % cos_dist_max)
print()

# 入力発話　その２
input_data = '最近面白かったものは何ですか'
input_data_mecab = parse_mecab(input_data)

response, cos_dist_max = matching_bagofwords(input_data_mecab, pair_data_mecab)

print('入力：%s' % input_data)
print('応答：%s' % response)
print('類似度：%.3f' % cos_dist_max)

入力：趣味は何ですか
応答：趣味はスポーツ観戦です
類似度：1.000

入力：最近面白かったものは何ですか
応答：最近食べたのはラーメンです
類似度：0.577


In [8]:
# 学習済みWord2vecファイルを読み込む
model_filename = './data/entity_vector.model.bin'
model_w2v = KeyedVectors.load_word2vec_format(model_filename, binary=True)

# 単語ベクトルの次元数
print(model_w2v.vector_size)

200


In [9]:
# Word2vecで特徴量を作成する関数を定義
# ここでは文内の各単語のWord2vecを足し合わせたものを文ベクトルととして利用する
# ※言語理解のときと同じ関数
def make_sentence_vec_with_w2v(words, model_w2v):

    sentence_vec = np.zeros(model_w2v.vector_size)
    num_valid_word = 0
    for w in words:
        if w in model_w2v:
            sentence_vec += model_w2v[w]
            num_valid_word += 1
    
    # 有効な単語数で割
    sentence_vec /= num_valid_word
    return sentence_vec


# 試しに変換してみる
feature_vec = make_sentence_vec_with_w2v(pair_data_mecab[0][0], model_w2v)
print(feature_vec)

[-1.14124946e-01 -2.45451868e-01  3.18230629e-01 -2.42564693e-01
 -1.68483868e-01  1.18045664e+00 -7.67753646e-02  1.78503878e-02
 -4.62107748e-01 -1.85864642e-01 -1.13828540e-01 -3.55474725e-02
 -1.10299480e+00  1.73642501e-01  1.47907531e+00  3.82385731e-01
  1.00570440e+00 -2.98919212e-02  1.84403554e-01 -4.12021726e-01
  1.99040741e-01  4.81746383e-02 -6.42520070e-01 -7.06509352e-01
  1.35463178e-01  6.30207583e-02  3.04609854e-02  5.22573948e-01
 -1.09154820e+00  8.61533821e-01 -1.86385915e-01  7.11009443e-01
  7.48171151e-01  7.42576182e-01  1.00709951e+00 -6.53035402e-01
  1.04251623e-01 -1.80611953e-01  1.19915617e+00  1.05364037e+00
  7.66198158e-01 -7.74398088e-01  1.82870805e-01  6.92205364e-03
  7.19385505e-01  9.12522078e-01 -3.59360665e-01 -6.56168461e-01
  3.60351093e-02  2.89187372e-01 -9.25456583e-01 -6.83573127e-01
  4.35496598e-01  5.24533033e-01  5.94134629e-01  7.08352804e-01
  2.84384400e-01 -8.63615870e-01  8.50027978e-01  1.75993371e+00
  5.42705320e-02 -5.33843

## 類似度計算

では、類似度を計算する関数のWord2vec版を用意しましょう。
入出の仕様は先ほどと同じで、入力ユーザ発話の単語の系列と、用例データを受け取り、入力ユーザ発話に最も類似するシステム応答を返します。

In [10]:
# 類似度計算（Word2vec版）
# 入力：ユーザ発話の単語の系列と用例データ
# 出力：入力ユーザ発話に最も類似するシステム応答
def matching_word2vec(input_data_mecab, pair_data_mecab):
    
    # コサイン類似度が最も高いものを採用
    cos_dist_max = 0.
    response = None
    
    # 用例毎に処理
    for pair_each in pair_data_mecab:
        
        # Bag-of-Words表現に変換
        v1 = make_sentence_vec_with_w2v(input_data_mecab, model_w2v)
        v2 = make_sentence_vec_with_w2v(pair_each[0], model_w2v)
        
        # コサイン類似度を計算
        cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        if cos_dist_max < cos_sim:
            cos_dist_max = cos_sim
            response = pair_each[1]
    
    return response, cos_dist_max
        

## テスト（Word2vec）
では、Word2vecを用いた用例ベース対話システムをテストしてみましょう。

In [18]:
# テスト

# 入力発話　その１
input_data = '趣味は何ですか'
input_data_mecab = parse_mecab(input_data)

response, cos_dist_max = matching_word2vec(input_data_mecab, pair_data_mecab)

print('入力：%s' % input_data)
print('応答：%s' % response)
print('類似度：%.3f' % cos_dist_max)
print()

# 入力発話　その２
input_data = '最近面白かったものは何ですか'
input_data_mecab = parse_mecab(input_data)

response, cos_dist_max = matching_word2vec(input_data_mecab, pair_data_mecab)

print('入力：%s' % input_data)
print('応答：%s' % response)
print('類似度：%.3f' % cos_dist_max)

入力：趣味は何ですか
応答：趣味はスポーツ観戦です
類似度：1.000

入力：最近面白かったものは何ですか
応答：最近は映画鑑賞にはまっています
類似度：0.826
